In [8]:
import importlib

trainFileName = './data/News Classification Dataset/train.csv'
testFileName = './data/News Classification Dataset/test.csv'

# Training Embedding Models
## Word2Vec

In [9]:
import word_vectorization.models.word2vec.Word2Vec as Word2Vec
importlib.reload(Word2Vec)

word2VecModel = Word2Vec.Word2Vec(2, trainFileName, embeddingSize=300, k=3)
word2VecEmbeddings = word2VecModel.train(epochs=10, lr=0.005, batchSize=2**12, verbose=True)

Loaded training data. Vocabulary size: 43605
Found and loaded trained embeddings.


## SVD

In [10]:
import word_vectorization.models.svd.SVD as SVD
importlib.reload(SVD)

svdModel = SVD.SvdWordVectorizationModel(3, trainFileName, embeddingSize=300)
svdEmbeddings = svdModel.train()

Loaded training data. Vocabulary size: 43605
Model already trained. Embeddings loaded.


# Sentence Classification


In [11]:
import word_vectorization.classification.LstmClassifier as LstmClassifier
import word_vectorization.datasets.ClassificationDataset as ClassificationDataset
importlib.reload(LstmClassifier)
importlib.reload(ClassificationDataset)

classifierHyperParams = {'hiddenSize': 256, 'numLayers': 3, 'bidirectional': True, 'hiddenLayers': [128, 64], 'activation': 'tanh'}

## Word2Vec
We will use the embeddings obtained from Word2Vec for classification.

In [12]:
word2VecClassifier = LstmClassifier.SentenceClassifier(trainFileName,
                                                       word2VecModel,
                                                       **classifierHyperParams)
word2VecClassifier.train(epochs=4,
                         lr=0.001,
                         batchSize=32,
                         verbose=True)

Saved model not found or retrain flag is set. Starting training from scratch.


Training: 100%|██████████| 3750/3750 [34:48<00:00,  1.80it/s]


Epoch 1/4 | Loss: 0.867


Training: 100%|██████████| 3750/3750 [29:08<00:00,  2.14it/s]


Epoch 2/4 | Loss: 0.359


Training:  21%|██        | 793/3750 [05:37<23:02,  2.14it/s]

## SVD
We will use the embeddings obtained from SVD for classification.

In [ ]:
svdClassifier = LstmClassifier.SentenceClassifier(trainFileName,
                                                  svdModel,
                                                  **classifierHyperParams)
svdClassifier.train(epochs=4,
                    lr=0.001,
                    batchSize=32,
                    verbose=True)

Saved model not found or retrain flag is set. Starting training from scratch.


/home/ashmitchamoli/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Training:  17%|█▋        | 645/3750 [07:04<34:04,  1.52it/s]  


KeyboardInterrupt: 

# Testing

In [ ]:
trainDataset = ClassificationDataset.ClassificationDataset(trainFileName, word2VecModel.wordIndices)
testDataset = ClassificationDataset.ClassificationDataset(testFileName, word2VecModel.wordIndices)

NameError: name 'ClassificationDataset' is not defined

## Word2Vec Classifier
### Test set evaluation

In [ ]:
word2VecScores = word2VecClassifier.evaluate(testDataset)
print(word2VecScores['Report'])

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      1900
           1       0.96      0.97      0.96      1900
           2       0.87      0.85      0.86      1900
           3       0.84      0.89      0.87      1900

    accuracy                           0.90      7600
   macro avg       0.90      0.90      0.90      7600
weighted avg       0.90      0.90      0.90      7600



### Train set evaluation

## SVD Classifier

In [ ]:
svdScores = svdClassifier.evaluate(testDataset)
print(svdScores['Report'])

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1900
           1       0.00      0.00      0.00      1900
           2       0.00      0.00      0.00      1900
           3       0.25      1.00      0.40      1900

    accuracy                           0.25      7600
   macro avg       0.06      0.25      0.10      7600
weighted avg       0.06      0.25      0.10      7600



# Hyperparameter Tuning
We will try different context sizes for each model.

## Training word embeddings

In [ ]:
contextSizes = [ 2, 3, 4, 5 ]

word2VecModels = []
svdModels = []

for contextSize in contextSizes:
    w2vModel = Word2Vec.Word2Vec(contextSize, trainFileName, embeddingSize=300, k=3)
    w2vEmbeddings = w2vModel.train(epochs=10, lr=0.005, batchSize=2**12, verbose=True)
    word2VecModels.append(w2vModel)

    svdModel = SVD.SvdWordVectorizationModel(3, trainFileName, embeddingSize=300)
    svdEmbeddings = svdModel.train()
    svdModels.append(svdModel)

## Training Sentence Classifiers

In [ ]:
trainFileName = './data/News Classification Dataset/train.csv'
w2vScores = {} # context size : scores
svdScores = {} # context size : scores

# compare the models
for w2vModel in word2VecModels:
    classifier = LstmClassifier.SentenceClassifier(trainFileName,
                                                   w2vModel,
                                                   **classifierHyperParams)
    classifier.train(epochs=5, lr=0.001, batchSize=32, verbose=True)

    w2vScores[w2vModel.contextSize] = classifier.evaluate(testDataset)

for svdModel in svdModels:
    classifier = LstmClassifier.SentenceClassifier(trainFileName,
                                                   svdModel,
                                                   **classifierHyperParams)
    classifier.train(epochs=5, lr=0.001, batchSize=32, verbose=True)

    svdScores[svdModel.contextSize] = classifier.evaluate(testDataset)

## Ranking the models

In [ ]:
# rank the models
from tabulate import tabulate

# sort according to f1 scores
sortedW2vScores = sorted(w2vScores.items(), key=lambda x: x[1]['F1'], reverse=True)
sortedSvdScores = sorted(svdScores.items(), key=lambda x: x[1]['F1'], reverse=True)

print(tabulate(sortedW2vScores, headers=['Context Size', 'Scores']))
print(tabulate(sortedSvdScores, headers=['Context Size', 'Scores']))